### Extract Log from Cloudant DB 
Author : Smart Dubai Govt - Manish


----

In [3]:
#!pip install cloudant

In [4]:
from cloudant.client import Cloudant
import cloudant as cld
from dateutil import tz
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey
import datetime
import time
import json
import csv
import pandas as pd

# Variables Setup

In [19]:
environment = 2              # UAT: 1, Dev : 0
language = 'EN' 
file_directory = 'D:\\4. Technical\\Watson Projects\\Logs\\' 
from_datetime = datetime.datetime(2018,3,1,0,0,0) # time in GMT  Format YYYY, MM , DD , HH,MM, SS
to_datetime   = datetime.datetime(2018,3,31,23,59,0)
cloudant_db = "logs-data"


In [20]:
filedate_name = datetime.datetime.now().strftime("%Y%m%d%H%M")
if language =='AR':
    log_input_path =  file_directory  + 'LOG_CLOUDANT_PROD_AR_'+ filedate_name +'.csv'
else: 
    log_input_path =  file_directory  + 'LOG_CLOUDANT_PROD_EN_'+ filedate_name +'.csv'




In [21]:
#Convert Date to UNIX Date format
epoch = datetime.datetime(1970,1,1)
from_datetime = (from_datetime - epoch).total_seconds()*1000
to_datetime = (to_datetime - epoch).total_seconds()*1000

# Convert epoch/unix time to datetime
#time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(1508756400.0))

In [22]:
# METHOD 1: Hardcode zones:
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('Asia/Dubai')

# METHOD 2: Auto-detect zones:
#from_zone = tz.tzutc()
#to_zone = tz.tzlocal()

def convertToDateTime(df , colnames):  # col name must be provided as a list
    for col in colnames:
        df[col] = pd.to_datetime(df[col])
    return df

def convertUtcToLocal(df , colnames):
    for col in colnames:
        df[col]=df[col].dt.tz_localize('utc').dt.tz_convert(to_zone).astype(str)
        df[col]= df[col].map(lambda x: x.replace('+04:00','0')).map(lambda x: x[0:19])
        df[col]=  pd.to_datetime(df[col])
    return df    

In [23]:
# Use the Cloudant library to create a Cloudant client.
client = Cloudant(serviceUsername, servicePassword, url=serviceURL)
client.connect()

# Create an instance of the database.
myDatabase = client[cloudant_db]   # client.create_database(databaseName)
if myDatabase.exists:
    rows = []
    result_collection = Result(myDatabase.all_docs, include_docs=True)
    query = cld.query.Query(myDatabase, sort =[{"_id": "asc"}],
        selector= {"$and": [ {"requestTime": {"$gte": from_datetime,"$lte": to_datetime}},{"language": language}]},
        fields= ["language","conversationId","request.workspace_id","request.requestTime"
                ,"response.intents", "response.input.text","response.output.text"]
        )

    #print(query)
    for doc in query()['docs']:   # for doc in query(limit=100, skip=100)['docs']:
        row = {}
        try:
            row['Language'] = doc['language']
            row['conversation_id'] = doc['conversationId']
            row['workspace_id'] =  doc['request']['workspace_id']
            
            #a= doc['request']['requestTime']
            row['request TS'] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(doc['request']['requestTime']/1000.0))
            
            
            
            response = doc['response']
            
            if len(response['intents']) > 0:
                row['Confidence'] = response['intents'][0]['confidence']
                row['Intent'] = response['intents'][0]['intent']
                
             
            if 'text' in response['input']: row['User Input'] = response['input']['text']
            for outtext in response['output']['text']:
                  if(len(outtext)>0):
                    row['Output'] = outtext

            #if 'text' in response['output']:row['Output'] = ' '.join(str(v) for v in response['output']['text'])


            rows.append(row)
        except Exception as e: 
            pass
  
    df_watson_log = pd.DataFrame(rows,columns=['Language','workspace_id','conversation_id','request TS','User Input','Output','Intent','Confidence'])
    df_watson_log= df_watson_log[df_watson_log['User Input'] != 'start']
    #df_watson_log.dropna(subset=['User Input'], how='all')
    df_watson_log = convertToDateTime(df_watson_log , ['request TS'])
    #Convert the time to Dubai time 
    df_watson_log = convertUtcToLocal(df_watson_log , ['request TS']) 
    df_watson_log = df_watson_log.sort_values(['conversation_id', 'request TS'], ascending=[False, False])
    #df_watson_log.to_csv(log_input_path, index = False,encoding='utf-8')
    
client.disconnect()


In [7]:
print('Execution completed :'  + datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

Execution completed :2018-02-01 14:00
